In [1]:
import torch
import math
import numpy as np
from visdom import Visdom
import time
torch.__version__

'1.0.0'

# 4.2.1 Use Visdom to visualize in PyTorch
Visdom is a visualization tool for PyTorch released by Facebook in 2017. [Official website](https://github.com/facebookresearch/visdom), visdom is generally defined as server-side matplot due to its simple function, which means that we can directly use the python console mode for development and on the server Execute, transfer some visual data to Visdom service, and visualize through Visdom service

## Installation
The installation of Visdom is very simple, just use the command `pip install visdom` to install it.
After the installation is complete, use the command `python -m visdom.server` to start the server locally, and it will prompt `It's Alive! You can navigate to http://localhost:8097` after startup. This means that the service is available. Browser, enter `http://localhost:8097` to see the page.

Port 8097 is the default port. You can specify the port by adding the `-port` parameter after the startup command. Commonly used parameters include `--hostname`, `-base_url`, etc.

## pit
Visdom's service will automatically download some static files when it is started. Here comes the pit. For some reasons that cannot be described, the download will fail, such as a prompt like this `ERROR:root:Error 404 while downloading https:// unpkg.com/layout-bin-packer@1.4.0` means that the static file has not been downloaded completely, so it may not be opened or there is no menu bar in the page, then you need to download it manually. Copy the static files directly to `Lib\site-packages\visdom`.

If you don’t know where the conda environment directory is, you can use `conda env list` to view

Thanks to the missing files provided by the partners of CSDN, the original text [here](https://blog.csdn.net/qq_36941368/article/details/82288154)

## basic concepts
### Environments
The role of Environments is to partition the visualization area. Each user will have a default partition called main, as shown in the figure:
![](1.PNG)
In the case specified by the program, the default chart will be placed here

## Panes
Panes is used as a container for each visualization chart. It can be filled with generated charts, pictures, and text. We can drag and drop, delete, resize and destroy Panes:
![](2.PNG)
Panes and Environments have a one-to-many relationship, that is, an Environment can contain multiple Panes


### VIEW
After adjusting Panes, the status can be managed through VIEW:
![](3.PNG)

## Visual interface
Visdom is the visualization support provided by Plotly, so provide the following visualization interface:
-vis.scatter: 2D or 3D scatter chart
-vis.line: line graph
-vis.stem: Stem and leaf diagram
-vis.heatmap: Heat map
-vis.bar: bar graph
-vis.histogram: histogram
-vis.boxplot: box plot
-vis.surf: surface map
-vis.contour: contour map
-vis.quiver: draw a two-dimensional vector field
-vis.image: image
-vis.text: text
-vis.mesh: mesh map
-vis.save: serialization status

## Use
### Draw simple graphics
Here we use the official DEMO as an example

In [2]:
env = Visdom()
assert env.check_connection() #Test the link, if the link is wrong, an error will be reported

Here generate two curve data sin and cos

In [3]:
Y = np.linspace(0, 2 * math.pi, 70)
X = np.column_stack((np.sin(Y), np.cos(Y)))

Use stem and leaf diagram to display

In [4]:
env.stem(
        X=X,
        Y=Y,
        opts=dict(legend=['Sine','Cosine'])
    )

'window_36f18bc34b4992'

Environments can be specified through the env parameter. If the name contains an underscore `_`, then visdom will be split and automatically grouped according to the underscore

In [5]:
envtest = Visdom(env='test_mesh')
assert envtest.check_connection()

Generate a grid map

In [6]:
x = [0, 0, 1, 1, 0, 0, 1, 1]
y = [0, 1, 1, 0, 0, 1, 1, 0]
z = [0, 0, 0, 0, 1, 1, 1, 1]
X = np.c_[x, y, z]
i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2]
j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3]
k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6]
Y = np.c_[i, j, k]
envtest.mesh(X=X, Y=Y, opts=dict(opacity=0.5))

'window_36f18bc533e990'

## Update loss function

During training, we will print the training loss and test accuracy in each batch. The graph displayed in this way needs to dynamically increase the data. Let's simulate this situation.

In [7]:
x,y=0,0
env2 = Visdom()
pane1= env2.line(
    X=np.array([x]),
    Y=np.array([y]),
    opts=dict(title='dynamic data'))

In [8]:
for i in range(10):
    time.sleep(1) #Print data every second
    x+=i
    y=(y+i)*1.5
    print(x,y)
    env2.line(
        X=np.array([x]),
        Y=np.array([y]),
        win=pane1,#win parameter confirms which pane to use
        update='append') #The action we do is append, there are other ways besides appending accidents, we will not introduce it here

0 0.0
1 1.5
3 5.25
6 12.375
10 24.5625
15 44.34375
21 75.515625
28 123.7734375
36 197.66015625
45 309.990234375


After running the above program, switch to visdom to see the effect

The basic usage of visdom is introduced, the next section introduces the more powerful tensorboardx